In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE127136"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE127136"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE127136.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE127136.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE127136.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Single-cell RNA-seq profiling reveals novel insights in immune-complex deposition and epithelium transition in IgA nephropathy"
!Series_summary	"IgA nephropathy represents the most prevalent chronic nephrosis worldwide. However, pathogenesis about IgA deposition and end-stage renal failure is still not well defined. Using single-cell RNA-seq, we identified the mesangial membrane receptor for IgA, which collaborates with increased extracellular matrix proteins and protease inhibitor to facilitate IgA deposition. Meanwhile, cell-cell interaction analysis revealed increased communications between mesangium and other cell types, uncovering how morbidity inside glomerulus spreads to whole kidney, which results in the genetic changes of kidney resident immune cells. Prominent interaction decreasing in intercalated cells leads to the discovery of a transitional cell type, which exhibited significant EMT and fibrosis features. Our work comprehensively cha

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine Gene Expression Data Availability
# Based on the background information, this dataset contains single-cell RNA-seq profiling data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: IgA nephropathy status can be inferred from row 1 'disease state'
trait_row = 1  

# Age: There is no information about age in the sample characteristics
age_row = None

# Gender: There is no information about gender in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary (0 for control, 1 for IgA nephropathy)"""
    if value is None:
        return None
    
    # The values in the sample characteristics dictionary are already in the format
    # "disease state: IgAN", so we need to extract the actual value
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.lower()
    
    # Based on the data, "igan" indicates having the disease
    if value == "igan":
        return 1
    # "normal" or "kidney cancer" indicates not having IgA nephropathy
    elif value in ["normal", "kidney cancer"]:
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to float (not used in this dataset)"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (not used in this dataset)"""
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Check if we have both control and case values
    unique_values = selected_clinical_df.iloc[0].unique()
    print(f"Unique trait values in the dataset: {unique_values}")
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM3625775': [1.0], 'GSM3625776': [1.0], 'GSM3625777': [1.0], 'GSM3625778': [1.0], 'GSM3625779': [1.0], 'GSM3625780': [1.0], 'GSM3625781': [1.0], 'GSM3625782': [1.0], 'GSM3625783': [1.0], 'GSM3625784': [1.0], 'GSM3625785': [1.0], 'GSM3625786': [1.0], 'GSM3625787': [1.0], 'GSM3625788': [1.0], 'GSM3625789': [1.0], 'GSM3625790': [1.0], 'GSM3625791': [1.0], 'GSM3625792': [1.0], 'GSM3625793': [1.0], 'GSM3625794': [1.0], 'GSM3625795': [1.0], 'GSM3625796': [1.0], 'GSM3625797': [1.0], 'GSM3625798': [1.0], 'GSM3625799': [1.0], 'GSM3625800': [1.0], 'GSM3625801': [1.0], 'GSM3625802': [1.0], 'GSM3625803': [1.0], 'GSM3625804': [1.0], 'GSM3625805': [1.0], 'GSM3625806': [1.0], 'GSM3625807': [1.0], 'GSM3625808': [1.0], 'GSM3625809': [1.0], 'GSM3625810': [1.0], 'GSM3625811': [1.0], 'GSM3625812': [1.0], 'GSM3625813': [1.0], 'GSM3625814': [1.0], 'GSM3625815': [1.0], 'GSM3625816': [1.0], 'GSM3625817': [1.0], 'GSM3625818': [1.0], 'GSM3625819': [1.0], 'GSM3625820': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")

SOFT file: ../../input/GEO/Chronic_kidney_disease/GSE127136/GSE127136_family.soft.gz
Matrix file: ../../input/GEO/Chronic_kidney_disease/GSE127136/GSE127136_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene expression data could not be successfully extracted from this dataset.
